# LLama demo

LlamaIndex 🦙 (GPT Index) is a project that provides a central interface to connect your large language models (LLMs) with external data. It allows you to index your data for various LLM tasks, such as text generation, summarization, question answering, etc., and remove concerns over prompt size limitations. It also supports data connectors to your common data sources and provides cost transparency and tools that reduce cost while increasing performance.

In [1]:
#%pip install openai --upgrade
#%pip install langchain --upgrade
#%pip install llama-index --upgrade

In [2]:
import os
import openai
import langchain
import llama_index
import nltk

from dotenv import load_dotenv
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, PromptHelper, LangchainEmbedding, ServiceContext
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [3]:
print('Open AI version:', openai.__version__)
print('langchain version:', langchain.__version__)
print('llama_index version:', llama_index.__version__)

Open AI version: 0.27.9
langchain version: 0.0.278
llama_index version: 0.8.8


In [4]:
# Load environment variables (set OPENAI_API_KEY, OPENAI_API_BASE, and OPENAI_API_VERSION in .env)
load_dotenv("azure.env")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_version = os.getenv('OPENAI_API_VERSION')

In [5]:
# Initialize LLM and Embeddings model (model is the actual model name, e.g., gpt-35-turbo, engine is your custom deployment name, e.g., my-gpt-35-turbo)
llm = AzureOpenAI(engine="gpt-35-turbo", model="gpt-35-turbo", temperature=0.0)
embeddings = LangchainEmbedding(OpenAIEmbeddings(deployment_id="text-embedding-ada-002", chunk_size=1))

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.
  warnings.warn(


In [6]:
llm

AzureOpenAI(callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f39f4746650>, model='gpt-35-turbo', temperature=0, max_tokens=None, additional_kwargs={}, max_retries=10, engine='gpt-35-turbo')

In [7]:
embeddings

LangchainEmbedding(model_name='text-embedding-ada-002', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f3a6c270640>)

In [8]:
# Define prompt helper
prompt_helper = PromptHelper(context_window=3000, 
                             num_output=500, 
                             chunk_overlap_ratio=0.1, 
                             chunk_size_limit=1000)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, prompt_helper=prompt_helper)

In [9]:
!ls docs/*.* -lh

-rwxrwxrwx 1 root root 6.6K Sep  5 09:15 docs/overview_clu.txt
-rwxrwxrwx 1 root root 8.9K Sep  5 09:15 docs/overview_openai.txt
-rwxrwxrwx 1 root root 3.6K Sep  5 09:15 docs/overview_translator.txt


In [10]:
# Load documents
documents = SimpleDirectoryReader('docs/').load_data()

# Create index
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, prompt_helper=prompt_helper)
query_engine = index.as_query_engine(service_context=service_context, verbose=True)

In [11]:
query = "What is azure openai service? give me back a bullet point list"
answer = query_engine.query(query)

print(f"Query: {query}")
print()
print(f"Answer: {answer}")
print()
print(f"Sources: {answer.get_formatted_sources()}")

Query: What is azure openai service? give me back a bullet point list

Answer: - Azure OpenAI is a service that provides REST API access to OpenAI's language models including GPT-3, Codex, and Embeddings series.
- The service offers several different models with varying capabilities and price points, including the GPT-3 base models Davinci, Curie, Babbage, and Ada, and the Codex series of models.
- Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio.
- The service provides a text-in, text-out interface where users can input a prompt and the model will generate a text completion.
- Azure OpenAI has virtual network support and managed identity via Azure Active Directory.
- The service is available in East US, South Central US, and West Europe regions.
- Content filtering is in place to evaluate prompts and completions against a content policy with automated systems to filter high severity content.

Sources: > Source (Doc id: b1d